In [1]:
import pandas as pd
import numpy as np

In [2]:
ttemp       = pd.read_csv("team_att_w_cluster.csv")  
ttemp['date']  = pd.to_datetime(ttemp['date'])  # add ending date/time for which team info is valid
team_att    = ttemp.assign(next_date=ttemp.groupby('team_id').date.shift(-1))
match       = pd.read_csv("match_discrete_htwt.csv")
match['date']  = pd.to_datetime(match['date'])
player_att  = pd.read_csv("player_att_with_endtime.csv")

cont_att    = [att for att in player_att.loc[:,'overall_rating':'next_date'].select_dtypes("float64")]
disc_att    = [ 'preferred_foot_right',
                'preferred_foot_left',
                'attacking_work_rate_medium',
                'attacking_work_rate_high',
                'attacking_work_rate_low',
                'defensive_work_rate_medium',
                'defensive_work_rate_high',
                'defensive_work_rate_low' ]
htwt        = ['height', 'weight']
atts        = cont_att + disc_att + htwt
h_atts      = ['h_' + lab for lab in atts]
a_atts      = ['a_' + lab for lab in atts]

In [3]:
def collate_info(row):
    date = row['next_date']
    h_matches = match[match['home_team_id'] == row['team_id']]
    h_matches = h_matches[(h_matches['date'] >= row['date']) & ((date != date) | (h_matches['date'] < date))]
    h_matches = h_matches[h_atts]
    h_matches.columns = atts
    a_matches = match[match['away_team_id'] == row['team_id']]
    a_matches = a_matches[(a_matches['date'] >= row['date']) & ((date != date) | (a_matches['date'] < date))]
    a_matches = a_matches[a_atts]
    a_matches.columns = atts
    res = h_matches.append(a_matches, ignore_index=True).mean(axis=0)
    return res

In [4]:
team_att[atts] = team_att.apply(collate_info, axis=1, result_type='expand')
team_att.to_csv("team_att_cluster_endtime_collated.csv")